# Основные точки руки

## Hands модель - множество рук
Данная модель позволяет находить множество рук но не различает правую и левую руки:

In [2]:
#pip install opencv-python mediapipe msvc-runtime
import mediapipe as mp
import time
import cv2

import random
import numpy as np
import matplotlib.pyplot as plt

<p align="center">
  <img src="images/hand_landmarks.png" alt="точки тела">
</p>


Определение поднят большой палец вверх или опущен вниз для всех имеющихся рук на видео:

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Инициализация Mediapipe Hands
hands = mp_hands.Hands(max_num_hands=3, min_detection_confidence=0.75,
                           min_tracking_confidence=0.5,)

cap = cv2.VideoCapture(r"C:\Users\ivano\Desktop\47dd5df0-6fab-4d01-b238-2bc8e10d0cd8.mp4")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

res_arr = []
while cap.isOpened():
    ret, image = cap.read()
    if not ret:
        break

    # Перевод кадра в RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Обнаружение рук на кадре
    results = hands.process(rgb_image)


    # image = cv2.imread(r"C:\Users\ivano\Desktop\1626146553_52-kartinkin-com-p-chyornii-fon-bez-vsego-krasivo-62.png")
    # rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


    print(results.multi_hand_landmarks)
    res_arr.append(results.multi_hand_landmarks)


    # Отрисовка результатов на кадре
    if results.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            # Отрисовка точек руки
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Позиция большого пальца (4-й элемент) правой руки
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            thumb_tip_x, thumb_tip_y = int(thumb_tip.x * image.shape[1]), int(thumb_tip.y * image.shape[0])

            # Позиция всех кончиков пальцев руки кроме - большого
            fingertips = [hand_landmarks.landmark[i] for i in [8,12,16,20]]  
            fingertips_x = [int(point.x * image.shape[1]) for point in fingertips]
            fingertips_y = [int(point.y * image.shape[0]) for point in fingertips]

            # Поднят ли большой палец (он выше всех других и оттапырен > 0.85 ширины оствльных пальцев) 
            # последнее условие для отработки случаев когда палец вытянут а рука боком
            if (thumb_tip_y < min(fingertips_y) and 
                        abs(min(fingertips_y)-thumb_tip_y) > 0.85* (max(fingertips_y)-min(fingertips_y))
                        and abs((fingertips_x[2])-thumb_tip_x) < 0.85* (max(fingertips_y)-min(fingertips_y))):            
                class_symbol = ':)'
            # Поднят ли большой палец (он ниже всех других и оттапырен > 0.85 ширины оствльных пальцев)
            elif (thumb_tip_y > max(fingertips_y) and 
                        abs(max(fingertips_y)-thumb_tip_y) > 0.85*(max(fingertips_y)-min(fingertips_y))
                        and abs((fingertips_x[2])-thumb_tip_x) < 1.75* (max(fingertips_y)-min(fingertips_y))):
                class_symbol = ':('
            else:
                class_symbol = ':|'

            # Отрисовка символа класса
            cv2.putText(image, class_symbol, (50, (i*70) + 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 0), 2, cv2.LINE_AA)

    


    # Отображение кадра
    cv2.imshow('Hand Tracking', image)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

with open('hands.txt', 'w') as file:
    file.write(str(res_arr))


[landmark {
  x: 0.8102237
  y: 0.79140484
  z: 3.7899036e-07
}
landmark {
  x: 0.78734976
  y: 0.8053041
  z: -0.026054662
}
landmark {
  x: 0.7803524
  y: 0.8279455
  z: -0.030207306
}
landmark {
  x: 0.7736797
  y: 0.8472459
  z: -0.027809972
}
landmark {
  x: 0.76690304
  y: 0.86341935
  z: -0.02272592
}
landmark {
  x: 0.8146932
  y: 0.8393689
  z: -0.015506522
}
landmark {
  x: 0.7994395
  y: 0.8658042
  z: -0.016748019
}
landmark {
  x: 0.78079784
  y: 0.87616384
  z: -0.017000593
}
landmark {
  x: 0.76502126
  y: 0.88185996
  z: -0.015459997
}
landmark {
  x: 0.8215606
  y: 0.8393014
  z: 0.0018858506
}
landmark {
  x: 0.80740947
  y: 0.8642985
  z: 0.0037939327
}
landmark {
  x: 0.78866047
  y: 0.8726302
  z: 0.0039465977
}
landmark {
  x: 0.77164555
  y: 0.8766412
  z: 0.004512338
}
landmark {
  x: 0.8203528
  y: 0.83712757
  z: 0.017381644
}
landmark {
  x: 0.80742306
  y: 0.85792994
  z: 0.01995871
}
landmark {
  x: 0.7912512
  y: 0.8658038
  z: 0.017794916
}
landmark {
  x

In [3]:
results.multi_hand_landmarks

## Holistic модель - различаем правую и левую
Оценим результат эмоции лишь по правой руке

## Holistic модель - сразу все находим (руки, тело и лицо)

In [3]:
def calculate_average_fps(current_fps, fps_buffer):
    # Добавляем текущее значение в буфер
    fps_buffer.pop(0)
    fps_buffer.append(current_fps)

    # Вычисляем среднее значение в буфере
    average_fps = sum(fps_buffer) / len(fps_buffer)

    return average_fps, fps_buffer

In [ ]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
	min_detection_confidence=0.75,
	min_tracking_confidence=0.5
)

# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

drawing_spec_lines_r = mp_drawing.DrawingSpec(  # линии соединения
          color=(255,0,0),
          thickness=2,
          circle_radius=1
        )

drawing_spec_lines_l = mp_drawing.DrawingSpec(  # линии соединения
          color=(0,255,0),
          thickness=2,
          circle_radius=1
        )

# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture('../../../../../Desktop/SBER/slovo/train/00019bad-4c36-4cc5-a940-2f994bc4037a.mp4')
 
# Initializing current time and precious time for calculating the FPS
previousTime = 0
currentTime = 0
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

N = 15 # окно усреднения fps
fps_buffer = [0] * N  # Инициализация массива предыдущих значений
 
while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
     # Create a black image
    image = np.zeros(image.shape, np.uint8)

    # Drawing the Facial Landmarks
    mp_drawing.draw_landmarks(
      image,
      results.face_landmarks,
      mp_holistic.FACEMESH_CONTOURS,
      mp_drawing.DrawingSpec(
        color=(255,0,255),
        thickness=1,
        circle_radius=1
      ),
      mp_drawing.DrawingSpec(
        color=(0,255,255),
        thickness=1,
        circle_radius=1
      )
    )
 
    # Drawing Right hand Land Marks
    mp_drawing.draw_landmarks(
      image, 
      results.right_hand_landmarks, 
      mp_holistic.HAND_CONNECTIONS,
      connection_drawing_spec=drawing_spec_lines_r
    )
 
    # Drawing Left hand Land Marks
    mp_drawing.draw_landmarks(
      image, 
      results.left_hand_landmarks, 
      mp_holistic.HAND_CONNECTIONS,
      connection_drawing_spec=drawing_spec_lines_l
    )

    # Drawing pose Marks
    mp_drawing.draw_landmarks(
      image, 
      results.pose_landmarks, 
      mp_holistic.POSE_CONNECTIONS
    )

     
    # Calculating the FPS
    currentTime = time.time()
    fps = 1 / (currentTime-previousTime)
    previousTime = currentTime

    fps, fps_buffer = calculate_average_fps(fps, fps_buffer)
     
    # Displaying FPS on the image
    cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (255,100,0), 2)
 
    # Display the resulting image
    cv2.imshow("Facial and Hand Landmarks", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\ProgramData\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\ProgramData\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionRese

: 